# Variable $R_{t}$ Sampler

Mixed sampling routine for sythentic model data with variable $R_{t}$. This is a vector form (with a difference value for each element of the time series), and will be inferred alongside the true series values $I_{t}$.

A periodic reporting trend is achieved by assigning each weekday a fixed reporting bias $\alpha_{w[t]}$, which does not vary over time and is independant of other days. This scales the true case/death number on each day to give the recorded count.

## Fixed Bias Formulation

Observed (biased) case values are sampled from a Poisson distribution: $C_{t} \sim Po(\alpha_{w[t]} I_{t})$, where the true cases are still given by the renewal equation $I_{t} \sim Po(R_{t} \Lambda_{t})$. Note that this model is not mechanistic, and there is no requirement imposed that the number of cases is conserved across the reporting process (i.e. that valules of $\alpha_{i}$ average to unity), although the inference process tends not to deviate far from this.

The parameters are updated in three steps. 

**Firstly** the reproduction value $R_{t}$ is sampled for every timestep $t$. We use a sliding window to assume $R_{t}$ has been constant for the previous 7 days. This gives the following conditional probability (derived in the next section), based on the prior $Gamma(a_{R}, b_{R})$:

$$
P(R_{t} | I_{t}) \propto Gamma \Biggl(a_{R} + \sum_{t'=t-6}^{t} I_{t'}, b_{R} + \sum_{t'=t-6}^{t}\Lambda_{t'} \Biggr) 
$$


In the first seven days, we will simply use the complete history without worrying that there is less information available.

**Secondly** each time step $I_{t}$ is sampled independently from the following distribution:

$$
P(I_{t} | C_{t}, \alpha_{i}, R_{t}) =  Po(I_{t} | R_{t} \Lambda_{t}) \times Po(C_{t} | \alpha_{i}I_{t})
$$

for discrete values of $I_{t}$, and where the $R_{t}$ estimated in the prior step is used.

N.B For $t=0$, $\Lambda_{t}$ is not defined, and therefore we take $\Lambda_{0} = C_{0}\alpha_{0}$ as an estimate of the seed cases instead during inference.

**Finally**, $\alpha_{i}$ are updated using Gibbs sampling from the following conditional probability:

$$
P(\alpha_{i} | I_{t} , C_{t}) \sim Gamma \left(a_{\alpha} + \sum_{t=i}^{n}C_{t}, b_{\alpha} + \sum_{t=i}^{n}I_{t} \right)
$$

(based on a gamma prior $Gamma(\alpha_{i} | a_{\alpha}, b_{\alpha})$ - see `fixed_bias_sampler.ipynb` for details of this derivation).






## Sampling the $R_{t}$ values

We use a sliding window to assume R_{t} has been constant for the previous 7 days, and compute the probability of a given $R_{t}$ value based on data from this period. This gives a conditional posterior distribution of the following form:

$$
P(R_{t} | I_{t}) = \prod_{t'=max(0, t-6)}^{t}\biggl[ P(I_{t'} | R_{t}) \biggr] P(R_{t})
$$

where we have taken a product over the previous seven days. Note that $R_{t}$ is still used for terms where $t' < t$ (instead of using other parameter values $R_{t-1}$ etc), as we have assumed that $R_{t}$ is constant over this period.

In the first seven days, we will use the complete history without worrying that there is less information available. For notational simplicity however, I will assume the summation begins at $t-6$ in all subsequent steps.

Based on the Poisson form of $P(I_{t'} | R_{t})$, we choose a conjugate gamma prior $Gamma(R_{t} | a_{R}, b_{R})$ for $R_{t}$:

$$
P(R_{t} | I_{t}) = \prod_{t'=t-6}^{t}\biggl[ Po(I_{t'} | R_{t} \Lambda_{t'}) \biggr] \times Gamma(R_{t} | a_{R}, b_{R})
$$

which we may write explicitly as :

$$
P(R_{t} | I_{t}) = \prod_{t'=t-6}^{t}\biggl[(R_{t}\Lambda_{t'})^{I_{t'}} \frac{e^{-R_{t}\Lambda_{t'}}}{I_{t'}!}  \biggr] \times R_{t}^{a_{R}-1} e^{-b_{R} R_{t}} \frac{b_{R}^{a_{R}}}{\Gamma(a_{R})}
$$

While the factorial $I_{t'}!$ is given by another parameter value, it is independant of $R_{t}$, and so may be neglected when formulating a distribution to sample from. Similarly, we may remove the multiplicative constant $\prod_{t'=t-6}^{t}\Lambda_{t'}^{I_{t'}}$. Simplifying the product we obtain:


$$
P(R_{t} | I_{t}) = \biggl[R_{t}^{\sum_{t'=t-6}^{t}I_{t'}} e^{-R_{t}\sum_{t'=t-6}^{t}\Lambda_{t'}} \biggr] \times R_{t}^{a_{R}-1} e^{-b_{R} R_{t}} \frac{b_{R}^{a_{R}}}{\Gamma(a_{R})}
$$

Combining like terms from the Poisson and Gamma distributions, and neglecting the constant terms in the Gamma distribution, we arrive at:


$$
P(R_{t} | I_{t}) \propto  R_{t}^{\sum_{t'=t-6}^{t}[I_{t'}] + a_{R}-1} \times e^{-R_{t}(\sum_{t'=t-6}^{t}[\Lambda_{t'}] + b_{R})}
$$


which we may write in the form of a Gamma distribution:


$$
P(R_{t} | I_{t}) \propto Gamma \Biggl(a_{R} + \sum_{t'=t-6}^{t} I_{t'}, b_{R} + \sum_{t'=t-6}^{t}\Lambda_{t'} \Biggr) 
$$



